<a href="https://colab.research.google.com/github/2021aim1014/K-Means-Clustering/blob/main/K_Means_Clustering_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from numpy.random import uniform
from sklearn.datasets import make_blobs
import seaborn as sns
import random

In [ ]:
def euclidean(point, data):
    return np.sqrt(np.sum((point - data)**2, axis=1))

In [ ]:
class KMeans:
    def __init__(self, n_clusters=8, max_iter=300):
        self.n_clusters = n_clusters
        self.max_iter = max_iter

    def fit(self, X_train):
        # Initialize the centroids, using the "k-means++" method, where a random datapoint is selected as the first,
        # then the rest are initialized w/ probabilities proportional to their distances to the first
        # Pick a random point from train data for first centroid
        self.centroids = [random.choice(X_train)]
        for _ in range(self.n_clusters-1):
            # Calculate distances from points to the centroids
            dists = np.sum([euclidean(centroid, X_train) for centroid in self.centroids], axis=0)
            # Normalize the distances
            dists /= np.sum(dists)
            # Choose remaining points based on their distances
            new_centroid_idx, = np.random.choice(range(len(X_train)), size=1, p=dists)
            self.centroids += [X_train[new_centroid_idx]]
        # This initial method of randomly selecting centroid starts is less effective
        # min_, max_ = np.min(X_train, axis=0), np.max(X_train, axis=0)
        # self.centroids = [uniform(min_, max_) for _ in range(self.n_clusters)]
        # Iterate, adjusting centroids until converged or until passed max_iter
        iteration = 0
        prev_centroids = None

        while np.not_equal(self.centroids, prev_centroids).any() and iteration < self.max_iter:
            # Sort each datapoint, assigning to nearest centroid
            sorted_points = [[] for _ in range(self.n_clusters)]
            for x in X_train:
                dists = euclidean(x, self.centroids)
                centroid_idx = np.argmin(dists)
                sorted_points[centroid_idx].append(x)
            # Push current centroids to previous, reassign centroids as mean of the points belonging to them
            prev_centroids = self.centroids
            self.centroids = [np.mean(cluster, axis=0) for cluster in sorted_points]
            for i, centroid in enumerate(self.centroids):
                if np.isnan(centroid).any():  # Catch any np.nans, resulting from a centroid having no points
                    self.centroids[i] = prev_centroids[i]
            iteration += 1
            print(iteration)
            self.inertia_ = np.sum([ np.min(dist)**2 for dist in self.transform(X_train)])

    def transform(self, X):
        # Transform to cluster-distance space
        # X: pd.DataFrame, independent variables, float
        # return dists = list of [dist to centroid 1, dist to centroid 2, ...]
        dists = [[self.dist(x,centroid) for centroid in self.centroids] for x in X]
        return dists

    def dist(self, point, data):
        return np.sqrt(np.sum((point - data)**2))
        
    def evaluate(self, X):
        centroids = []
        centroid_idxs = []
        for x in X:
            dists = euclidean(x, self.centroids)
            centroid_idx = np.argmin(dists)
            centroids.append(self.centroids[centroid_idx])
            centroid_idxs.append(centroid_idx)
        return centroids, centroid_idxs

In [ ]:
import tensorflow as tf
import numpy as np
import os

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
(x_train, y_train), (x_test, y_test) = (x_train[:5000], y_train[:5000]), (x_test[:5000], y_test[:5000])

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(5000, 28, 28)
(5000,)
(5000, 28, 28)
(5000,)


In [ ]:
# (X_train, Y_train), (X_test, Y_test) = (x_train[:100], y_train[:100]), (x_test[:100], y_test[:100])

In [ ]:
# print(X_train.shape)
# print(Y_train.shape)
# print(X_test.shape)
# print(Y_test.shape)

In [ ]:
import cv2

# defining feature extractor that we want to use
extractor = cv2.xfeatures2d.SIFT_create()

def compute_features(image):
    keypoints, descriptors = extractor.detectAndCompute(image, None)
    return keypoints, descriptors

In [ ]:
from tqdm import tqdm

In [ ]:
def CreateVisualDictionary(dataset):
  descriptor_list = []
  features = {}
  for idx, image in tqdm(enumerate(dataset)):
    k, d = compute_features(image)

    if d is not None:
      descriptor_list.extend(d)
      features[idx] = d
  return features, descriptor_list

In [ ]:
train_f, train_d = CreateVisualDictionary(x_train)
f_test, d_test = CreateVisualDictionary(x_test)

5000it [00:03, 1261.05it/s]
5000it [00:05, 868.03it/s] 


In [ ]:
# from sklearn.cluster import KMeans
def MatchHistogram(h1, h2):
    return np.sqrt(np.sum((h1 - h2)**2, axis=1))

In [ ]:
kmeans = KMeans(n_clusters=2)

In [ ]:
kmeans.fit(train_d)

In [ ]:
kmeans.inertia_

2445575836.74039

In [ ]:
bow = kmeans.centroids

In [ ]:
# add a function to save the sift of the image which is closest
def ComputeHistogram(features, bow):
  histograms = {}
  for img in tqdm(features):
    # all descriptors of the img
    descr = features[img]
    histogram = np.zeros(len(bow))
    class_centers, classification = kmeans.evaluate(descr)
    # for each des, find the cluster and create histogram
    for p in classification:
      p = int(p)
      histogram[p] += 1
    # update global histograms
    histograms[img] = histogram
  return histograms

In [ ]:
train_hist = ComputeHistogram(train_f, bow)

100%|██████████| 4601/4601 [00:01<00:00, 2542.95it/s]


In [ ]:
def prepare_dataset(histograms, y_train):
  trainX = []
  trainY = []

  for x in histograms:
    trainX.append(histograms[x])
    trainY.append(y_train[x])
  return trainX, trainY

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [ ]:
trainX, trainY = prepare_dataset(train_hist, y_train)

In [ ]:
clf = LinearSVC()
clf.fit(trainX, trainY)
preds = clf.predict(trainX)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
test_hist = ComputeHistogram(f_test, bow)

100%|██████████| 4641/4641 [00:01<00:00, 2385.99it/s]


In [ ]:
testX, testY = prepare_dataset(test_hist, y_test)

In [ ]:
preds_test = clf.predict(testX)
print(classification_report(testY, preds_test))

              precision    recall  f1-score   support

           0       0.61      0.55      0.58       480
           1       0.60      0.72      0.65       316
           2       0.41      0.46      0.43       496
           3       0.46      0.44      0.45       468
           4       0.41      0.46      0.43       506
           5       0.68      0.72      0.70       467
           6       0.28      0.14      0.19       445
           7       0.68      0.72      0.70       481
           8       0.62      0.64      0.63       507
           9       0.78      0.82      0.80       475

    accuracy                           0.56      4641
   macro avg       0.55      0.57      0.56      4641
weighted avg       0.55      0.56      0.55      4641



In [ ]:
testY = np.array(testY)

In [ ]:
preds_test = np.array(preds_test)

In [ ]:
np.mean(testY == preds_test)

0.5636716224951519